# MedConnect

In [20]:
pip install surprise

In [21]:
from surprise import Dataset, Reader, SVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd

In [22]:

dados = pd.read_csv("/content/fashion_products.csv")
dados.head(10)

,User ID,Product ID,Product Name,Brand,Category,Price,Rating,Color,Size
0,19,1,Dress,Adidas,Men's Fashion,40,1.043159,Black,XL
1,97,2,Shoes,H&M,Women's Fashion,82,4.026416,Black,L
2,25,3,Dress,Adidas,Women's Fashion,44,3.337938,Yellow,XL
3,57,4,Shoes,Zara,Men's Fashion,23,1.049523,White,S
4,79,5,T-shirt,Adidas,Men's Fashion,79,4.302773,Black,M
5,98,6,Dress,Adidas,Men's Fashion,47,1.379566,Yellow,L
6,16,7,Jeans,Gucci,Men's Fashion,37,1.356750,White,XL
7,63,8,Sweater,Zara,Kids' Fashion,64,4.360303,Blue,XL
8,96,9,Sweater,H&M,Men's Fashion,53,4.466182,Green,XL
9,36,10,T-shirt,Zara,Kids' Fashion,55,4.093234,White,XL


In [23]:
dados.isnull().sum()

User ID         0
Product ID      0
Product Name    0
Brand           0
Category        0
Price           0
Rating          0
Color           0
Size            0
dtype: int64

Criando dataset  para ser usado em Filtro baseado em conteudo


In [24]:
conteudo_df = dados[['Product ID', 'Product Name', 'Brand',
                   'Category', 'Color', 'Size']]
conteudo_df['Content'] = conteudo_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

<ipython-input-24-61b3e49cb969>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conteudo_df['Content'] = conteudo_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)


Vamos utilizar a função TfidfVectorizer() para fazer representações numericas dos valores em texto,
ultilizar a função linaer_kernel para avaliar a similaridade entre os produtos do dataset,
ultizar a função reader que permitirá que o programa interprete as interações de usuarios e produtos e e notas datas de tais usuarios para tais produtos.
Em seguida vamos criar uma função que retornará a recomenções de produtos baseados em suas notas



In [25]:

tfidf_vectorizer = TfidfVectorizer()
MatrizDeConteudo = tfidf_vectorizer.fit_transform(conteudo_df['Content'])

ConteudosSimilares = linear_kernel(MatrizDeConteudo, MatrizDeConteudo)

reader = Reader(rating_scale=(1, 5))
dados = Dataset.load_from_df(dados[['User ID',
                                  'Product ID',
                                  'Rating']], reader)

def get_content_based_recommendations(product_id, top_n):
    index = conteudo_df[conteudo_df['Product ID'] == product_id].index[0]
    similarity_scores = ConteudosSimilares[index]
    similar_indices = similarity_scores.argsort()[::-1][1:top_n + 1]
    recommendacoes = conteudo_df.loc[similar_indices, 'Product ID'].values
    return recommendacoes

In [26]:
get_content_based_recommendations(7,10)

array([972, 683, 266, 303, 223, 165, 310, 861, 941, 439])

Recomendação por filtro colaborativo

Vamos utilizar o algoritmo svd(Singular Value Decomposition) da biblioteca surprise, que é ultilizado especialmente para sistemas de recomendação colaborativa, para preparar o dataset escolhido para realizar o treino do modelo,
a função "build_anti_testset" será usada para gerar todas as combinações posiveis de produtos com os quais o usuario nunca interagiu, filtrar as linhas relativas ao usuario, osganiza-la do maior para o menor, fazer uma lista com os primeiras recomendações(top_n).

In [27]:

algoritmo = SVD()
trainset = dados.build_full_trainset()
algoritmo.fit(trainset)

def get_collaborative_filtering_recommendations(user_id, top_n):
    testset = trainset.build_anti_testset()
    testset = filter(lambda x: x[0] == user_id, testset)
    predictions = algoritmo.test(testset)
    predictions.sort(key=lambda x: x.est, reverse=True)
    recommendations = [prediction.iid for prediction in predictions[:top_n]]
    return recommendations

In [28]:
get_collaborative_filtering_recommendations(16,5)

[269, 145, 260, 491, 71]

O sistema de recomendação


Como ja realizamos todas as funções necessarias anteriormente, vamos criar uma função que recebe tanto id de produtos quanto id de usuarios e quantidade de recomendações de o sistema retornará para o usuario. vamos usar as funções dos metodos que ja criamos, combina-los e criar uma lista com as primeiras opções, com "set" para que não haja produtos duplicados


In [29]:

def get_hybrid_recommendations(user_id, product_id, top_n):
    content_based_recommendations = get_content_based_recommendations(product_id, top_n)
    collaborative_filtering_recommendations = get_collaborative_filtering_recommendations(user_id, top_n)
    hybrid_recommendations = list(set(content_based_recommendations + collaborative_filtering_recommendations))
    return hybrid_recommendations[:top_n]

    combined_recommendations = list(dict.fromkeys(collaborative_recommendations + content_recommendations))
    return combined_recommendations[:top_n]

In [31]:
user_id = 16
product_id = 7
top_n = 5
recommendacoes = get_hybrid_recommendations(user_id, product_id, top_n)

print(f"Hybrid Recommendations for User {user_id} based on Product {product_id}:")
for i, recommendation in enumerate(recommendacoes):
    print(f"{i + 1}. Product ID: {recommendation}")

Hybrid Recommendations for User 16 based on Product 7:
1. Product ID: 294
2. Product ID: 526
3. Product ID: 1241
4. Product ID: 794
5. Product ID: 828
